<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Neural-Style-Transfer" data-toc-modified-id="Neural-Style-Transfer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Neural Style Transfer</a></span><ul class="toc-item"><li><span><a href="#The-content-loss" data-toc-modified-id="The-content-loss-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>The content loss</a></span></li><li><span><a href="#The-style-loss" data-toc-modified-id="The-style-loss-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The style loss</a></span><ul class="toc-item"><li><span><a href="#In-short" data-toc-modified-id="In-short-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>In short</a></span></li></ul></li><li><span><a href="#Neural-style-transfer-in-Keras" data-toc-modified-id="Neural-style-transfer-in-Keras-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Neural style transfer in Keras</a></span></li><li><span><a href="#Defining-initial-variables" data-toc-modified-id="Defining-initial-variables-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Defining initial variables</a></span></li><li><span><a href="#Auxiliary-functions" data-toc-modified-id="Auxiliary-functions-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Auxiliary functions</a></span></li><li><span><a href="#Loading-the-pretrained-VGG19-network-and-applying-it-to-three-images" data-toc-modified-id="Loading-the-pretrained-VGG19-network-and-applying-it-to-three-images-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Loading the pretrained VGG19 network and applying it to three images</a></span></li><li><span><a href="#Content-Loss" data-toc-modified-id="Content-Loss-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Content Loss</a></span></li><li><span><a href="#Style-Loss" data-toc-modified-id="Style-Loss-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Style Loss</a></span></li><li><span><a href="#Total-variation-Loss" data-toc-modified-id="Total-variation-Loss-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Total variation Loss</a></span></li><li><span><a href="#Defining-the-final-loss-that-you-will-minimize" data-toc-modified-id="Defining-the-final-loss-that-you-will-minimize-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Defining the final loss that you will minimize</a></span></li></ul></li></ul></div>

# Neural Style Transfer

THe key notion behind implementing style transfer is the same idea that is central to all deep-learning algorithms:
- Define a loss function to specify what you want to achieve
- Minimize this loss

In this case, we want to conserve the content of the original image while adopting the style of the reference image. If we were able to mathematically define content and style, then an appropriate loss function to minimize would be the following:

`loss = distance(style(reference_image) - style(generated_image) +
        distance(content(original_image) - content(generated_image))`
        
Here, distance is a norm function such as the L2 norm, content is a function that takes an image and computes representation of its content, and style is a function that takes an image and computes a representation of its style. 

## The content loss

As you already know, activations from earlier layers in a network contain local information about the image, while activations from higher layers contain increasingly global and abstract information. Formulated in a different way, the activations of the different layers of a convnet provide a decomposition of the contents of an image over different spatial scales. Therefore we expect the "content" of an image, which is more global and more abstract, to be captured by the representations of a top layer of a convnet.

A good candidate for a content loss would thus be to consider a pre-trained convnet, and define as our loss the L2 norm between the activations of a top layer computed over the target image and the activations of the same layer computed over the generated image. This would guarantee that, as seen from the top layer of the convnet, the generated image will "look similar" to the original target image. Assuming that what the top layers of a convnet see is really the "content" of their input images, then this does work as a way to preserve image content.

## The style loss

While the content loss only leverages a single higher-up layer, the style loss as defined in the Gatys et al. paper leverages multiple layers of a convnet: we aim at capturing the appearance of the style reference image at all spatial scales extracted by the convnet, not just any single scale.

For the style loss, the Gatys et al. paper leverages the "Gram matrix" of a layer's activations, i.e. the inner product between the feature maps of a given layer. This inner product can be understood as representing a map of the correlations between the features of a layer. These feature correlations capture the statistics of the patterns of a particular spatial scale, which empirically corresponds to the appearance of the textures found at this scale.

Hence the style loss aims at preserving similar internal correlations within the activations of different layers, across the style reference image and the generated image. In turn, this guarantees that the textures found at different spatial scales will look similar across the style reference image and the generated image.


### In short

In short, we can use a pre-trained convnet to define a loss that will:

    Preserve content by maintaining similar high-level layer activations between the target content image and the generated image. The convnet should "see" both the target image and the generated image as "containing the same things".
    Preserve style by maintaining similar correlations within activations for both low-level layers and high-level layers. Indeed, feature correlations capture textures: the generated and the style reference image should share the same textures at different spatial scales.

Now let's take a look at a Keras implementation of the original 2015 neural style transfer algorithm. As you will see, it shares a lot of similarities with the Deep Dream implementation we developed in the previous section.

## Neural style transfer in Keras

Neural style transfer can be implemented using any pre-trained convnet. Here we will use the VGG19 network, used by Gatys et al in their paper. VGG19 is a simple variant of the VGG16 network we introduced in Chapter 5, with three more convolutional layers.

This is our general process:

    Set up a network that will compute VGG19 layer activations for the style reference image, the target image, and the generated image at the same time.
    Use the layer activations computed over these three images to define the loss function described above, which we will minimize in order to achieve style transfer.
    Set up a gradient descent process to minimize this loss function.

Let's start by defining the paths to the two images we consider: the style reference image and the target image. To make sure that all images processed share similar sizes (widely different sizes would make style transfer more difficult), we will later resize them all to a shared height of 400px.


!wget https://www.desktopbackground.org/download/o/2012/07/01/414337_imaginary-foundation-astronauts-colors-creativity-dreams-wallpapers_1920x1080_h.jpg

!wget https://hdwallsource.com/img/2014/1/colorado-wallpaper-15859-16345-hd-wallpapers.jpg

## Defining initial variables

In [4]:
from keras.preprocessing.image import load_img, img_to_array

In [5]:
target_image_path = 'assets/colorado-wallpaper-15859-16345-hd-wallpapers.jpg'
style_reference_image_path = 'assets/414337_imaginary-foundation-astronauts-colors-creativity-dreams-wallpapers_1920x1080_h.jpg'

In [6]:
width, height = load_img(target_image_path).size

In [7]:
width, height

(1920, 1200)

In [9]:
img_height = 400
img_width = int(width * img_height / height)
img_width

640

## Auxiliary functions


In [10]:
import numpy as np
from keras.applications import vgg19

In [11]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [12]:
def deprocess_image(x):
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x



Let's set up the VGG19 network. It takes as input a batch of three images: the style reference image, the target image, and a placeholder that will contain the generated image. A placeholder is simply a symbolic tensor, the values of which are provided externally via Numpy arrays. The style reference and target image are static, and thus defined using K.constant, while the values contained in the placeholder of the generated image will change over time.


## Loading the pretrained VGG19 network and applying it to three images

In [13]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# This placeholder will contain our generated image
combination_image = K.placeholder((1, img_height, img_width, 3))

# We combine the 3 images into a single batch
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# We build the VGG19 network with our batch of 3 images as input.
# The model will be loaded with pre-trained ImageNet weights.
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('Model loaded.')

80142336/80134624 [==============================] - 3s 0us/step
Model loaded.




Let's define the content loss, meant to make sure that the top layer of the VGG19 convnet will have a similar view of the target image and the generated image:


## Content Loss

In [14]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))



Now, here's the style loss. It leverages an auxiliary function to compute the Gram matrix of an input matrix, i.e. a map of the correlations found in the original feature matrix.


## Style Loss

In [15]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))



To these two loss components, we add a third one, the "total variation loss". It is meant to encourage spatial continuity in the generated image, thus avoiding overly pixelated results. You could interpret it as a regularization loss.


## Total variation Loss

In [16]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))



The loss that we minimize is a weighted average of these three losses. To compute the content loss, we only leverage one top layer, the block5_conv2 layer, while for the style loss we use a list of layers that spans both low-level and high-level layers. We add the total variation loss at the end.

Depending on the style reference image and content image you are using, you will likely want to tune the content_weight coefficient, the contribution of the content loss to the total loss. A higher content_weight means that the target content will be more recognizable in the generated image.


## Defining the final loss that you will minimize

In [17]:
# Dict mapping layer names to activation tensors
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# Name of layer used for content loss
content_layer = 'block5_conv2'
# Name of layers used for style loss
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Weights in the weighted average of the loss components
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# Define the loss by adding all components to a `loss` variable
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)



Finally, we set up the gradient descent process. In the original Gatys et al. paper, optimization is performed using the L-BFGS algorithm, so that is also what we will use here. This is a key difference from the Deep Dream example in the previous section. The L-BFGS algorithms comes packaged with SciPy. However, there are two slight limitations with the SciPy implementation:

- It requires to be passed the value of the loss function and the value of the gradients as two separate functions.
- It can only be applied to flat vectors, whereas we have a 3D image array.

It would be very inefficient for us to compute the value of the loss function and the value of gradients independently, since it would lead to a lot of redundant computation between the two. We would be almost twice slower than we could be by computing them jointly. To by-pass this, we set up a Python class named Evaluator that will compute both loss value and gradients value at once, will return the loss value when called the first time, and will cache the gradients for the next call.


In [18]:
# Get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)[0]

# Function to fetch the values of the current loss and the current gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

Finally, we can run the gradient ascent process using SciPy's L-BFGS algorithm, saving the current generated image at each iteration of the algorithm (here, a single iteration represents 20 steps of gradient ascent):

In [19]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 20

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Save current generated image
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0


ResourceExhaustedError: OOM when allocating tensor with shape[3,256,100,160]
	 [[Node: block3_conv4/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](block3_conv3/Relu, block3_conv4/kernel/read)]]
	 [[Node: gradients/AddN_16/_239 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1496_gradients/AddN_16", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'block3_conv4/convolution', defined at:
  File "/home/abanihi/anaconda3/envs/dl-gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/abanihi/anaconda3/envs/dl-gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/abanihi/anaconda3/envs/dl-gpu/lib/python3.6/site-packages/

In [20]:
%load_ext version_information
%version_information scipy, numpy, keras, matplotlib, tensorflow

Software versions
Python 3.6.4 64bit [GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
IPython 6.2.1
OS Linux 4.4.0 53 generic x86_64 with debian stretch sid
scipy 1.0.0
numpy 1.12.1
keras 2.0.9
matplotlib 2.1.1
tensorflow 1.3.0
Sun Jan 14 14:55:57 2018 CST